In [2]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import statistics

In [3]:
market = Market()
speculation_db = ADatabase("financial")

In [4]:
market.connect()
prices = market.retrieve("pdr_prices")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]

In [38]:
base_non_available = [{"start_date":datetime(2016,1,1),"end_date":datetime(2016,1,2),"ticker":"LOL"}]
positions = 10
start_date = datetime(2016,1,1)
end_date = datetime(2022,7,1)
reqs = [0.5]
signals = [0.5,1]
values = [True]
speculation_db.connect()
simulation = speculation_db.retrieve(f"pdr_sim")
simulation["year"] = [int(x) for x in simulation["year"]]
projection_value = 1
training_year = 4
sim = prices.copy().merge(simulation[(simulation["training_year"]==training_year) & \
                                    (simulation["projection_value"]==projection_value)],on=["year","quarter","ticker"],how="left")
sim = sim.dropna()
sim["prediction"] = (sim["xgb_prediction"] + sim["skl_prediction"] + sim["cat_prediction"]) / 3
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [39]:
delta_dict = []
for ticker in sim["ticker"].unique():
    ticker_data = sim[sim["ticker"]==ticker].sort_values("date")
    ticker_data = ticker_data.groupby(["year","quarter","ticker"]).mean().reset_index()
    ticker_data["model_delta"] = ticker_data["prediction"].pct_change(1)
    ticker_data.dropna(inplace=True)
    delta_dict.append(ticker_data[["year","quarter","ticker","model_delta"]])
deltas = pd.concat(delta_dict)

In [40]:
sim = sim.merge(deltas,on=["year","quarter","ticker"],how="left")

In [41]:
sim["model_delta"].mean()

-1.9212408691168388

In [42]:
for signal in tqdm(signals):
    for value in values:
        iterration_sim = sim.copy()
        if value==False:
            iterration_sim["delta"] = iterration_sim["delta"] * -1
        for req in reqs:
            not_available = [x for x in base_non_available]
            for position in range(positions):
                date = start_date
                nadf = pd.DataFrame(not_available)
                while date < end_date:
                    try:
                        year = date.year
                        todays_not_available = nadf[(nadf["start_date"]<=date) & (nadf["end_date"]>=date)]
                        todaynatickers = list(todays_not_available["ticker"].unique())
                        todays_recs = iterration_sim[(iterration_sim["date"]==date) & (~iterration_sim["ticker"].isin(todaynatickers))]
                        todays_recs.sort_values("delta",ascending=False,inplace=True)
                        if (todays_recs.index.size) > 0 and date.weekday() <= 2:
                            offering = todays_recs.iloc[position]
                            if (offering["delta"] >= signal) & (offering["model_delta"] > 0):
                                trade = offering
                                ticker = trade["ticker"]
                                buy_price = trade["adjclose"]
                                trade_req = req
                                exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)]
                                exits["gains"] = (exits["adjclose"] - buy_price) / buy_price
                                gain_exits = exits[(exits["gains"]>=trade_req) & (exits["quarter"]==trade["quarter"]) & (exits["year"]==trade["year"])].sort_values("date")
                                if gain_exits.index.size < 1:
                                    breakeven_exits = exits[(exits["gains"]>=0) & (exits["quarter"]>trade["quarter"]) & (exits["year"]==trade["year"])].sort_values("date")
                                    if breakeven_exits.index.size < 1:
                                        final_exits = exits[(exits["quarter"]>trade["quarter"]) & (exits["year"]==trade["year"])].sort_values("date")
                                        if final_exits.index.size < 1:
                                            exit = exits[(exits["year"]==trade["year"])].sort_values("date").iloc[-1]
                                            trade["sell_price"] = exit["adjclose"]
                                        else:
                                            exit = final_exits.iloc[0]
                                            trade["sell_price"] = exit["adjclose"]
                                    else:
                                        exit = breakeven_exits.iloc[0]
                                        trade["sell_price"] = buy_price
                                else:
                                    exit = gain_exits.iloc[0]
                                    trade["sell_price"] = buy_price * (1+(trade_req))
                                delta = (trade["sell_price"] - buy_price) / buy_price
                                trade["projected_delta"] = offering["delta"]
                                trade["sell_date"] = exit["date"]
                                trade["position"] = position
                                trade["req"] = req
                                trade["value"] = value
                                trade["delta"] = delta
                                trade["training_year"] = training_year
                                trade["projection_value"] = projection_value
                                trade["signal"] = signal
                                included_cols = ["date","sell_date","ticker","adjclose","sell_price","delta","projected_delta","position","req","value","training_year","signal","projection_value"]
                                trade_df = pd.DataFrame([trade])
                                speculation_db.store("pdr_hybrid_trades",trade_df[included_cols])
                                not_available.append({"start_date":trade["date"],"end_date":trade["sell_date"],"ticker":ticker})
                                date = exit["date"] + timedelta(days=1)                                     
                            else:
                                date = date + timedelta(days=1)
                        else:
                            date = date + timedelta(days=1)
                    except Exception as e:
                        print(str(e))
                        date = date+timedelta(days=1)
speculation_db.disconnect()

100%|███████████████████████████████████████████████████████| 2/2 [17:11<00:00, 515.71s/it]
